In [1]:
import enum
import logging
import os
import sys
import errno
import time
import math
import torch
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional
from tqdm import tqdm

from transformers import BertTokenizer, BertForMaskedLM, AdamW
import torch
import torch.optim as optim
from torch.utils.data import DataLoader 

import argparse

from src.genie.data import IEDataset, my_collate

C:\Users\steph\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
output_dir = 'out'
with_tracking = False

batch_size = 64
n_epochs = 1
weight_decay = 0.0
learning_rate = 5e-5

lr_scheduler_type = "linear"
num_warmup_steps = 0.0
max_train_steps = 10_000
gradient_accumulation_steps = 1
with_tracking = False

In [3]:
train = IEDataset('preprocessed_data/train_RAMS_oracle.jsonl')

In [4]:
train = DataLoader(train, 
    pin_memory=True, num_workers=2, 
    collate_fn=my_collate,
    batch_size=4, 
    shuffle=True)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_tokens(['<args>'])

model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30523, 768)

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(n_epochs):
    print('Starting Epoch:', epoch)
    for idx, data in enumerate(train):
        optimizer.zero_grad()
        input_ids = data['input_token_ids']
        labels = data['tgt_token_ids']
        attention_mask = data['input_attn_mask']
        print('Starting Forward:')
        loss, logits = model(input_ids, attention_mask=attention_mask, labels=labels)
        print('Starting Backward')
        loss.backward()
        optimizer.step()
        print('loss:')
        print(loss.item())


Starting Epoch: 0
Starting Forward:
Starting Backward
loss:
10.370660781860352
Starting Forward:
Starting Backward
loss:
9.486031532287598
Starting Forward:
Starting Backward
loss:
8.018885612487793
Starting Forward:
Starting Backward
loss:
6.058218955993652
Starting Forward:
Starting Backward
loss:
5.075306415557861
Starting Forward:
Starting Backward
loss:
5.07222318649292
Starting Forward:
Starting Backward
loss:
4.758015155792236
Starting Forward:
Starting Backward
loss:
4.114434242248535
Starting Forward:
Starting Backward
loss:
3.9111194610595703
Starting Forward:
Starting Backward
loss:
3.1673481464385986
Starting Forward:
Starting Backward
loss:
3.005366802215576
Starting Forward:
Starting Backward
loss:
2.6861190795898438
Starting Forward:
